# 단일 입력 PromptTemplate

In [1]:
from langchain.prompts import PromptTemplate

In [4]:
import langchain
langchain.__version__

'0.3.25'

In [7]:
# 템플릿 문자열 : 프롬프트의 골격을 정의
# PromptTemplate 객체 생성 : 인스턴스 변수 선언
# 포멧(format) 메서드 수행 : PromptTemplate.format()
# LLM 호출
template_str = (
  "당신은 최고 수준의 마케팅 카피라이터입니다. \n"
  "아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오. \n\n"
  "제품명: {product_name}"
)

prompt = PromptTemplate.from_template(template_str)

input_str = input("제품명을 입력하세요: ")
formatted_prompt = prompt.format(product_name=input_str)
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터입니다. 
아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오. 

제품명: 초슈퍼카메라 x100


In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

chain = prompt | llm

result = chain.invoke({"product_name": input_str})
print(result.content)

초슈퍼카메라 x100으로 순간을 선명하게! 뛰어난 화질과 빠른 반응속도로 최고의 사진을 경험하세요. 지금 바로 만나보세요!


# 다중 입력 PromptTemplate

In [10]:
multi_template_str = (
  "아래는 뉴스 기사 제목과 키워입니다.\n"
  "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오.\n\n"
  "제목 : {title}\n"
  "키워드 : {keywords}\n"
)

summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keywords"])

sample_title = "인공지능 기술의 발전과 미래"
sample_keywords = "머신러닝, 딥러닝, 산업 혁신"
formatted_summary_prompt = summary_prompt.format(title=sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오.

제목 : 인공지능 기술의 발전과 미래
키워드 : 머신러닝, 딥러닝, 산업 혁신



In [11]:
result_summary = (summary_prompt | llm).invoke({
    "title": sample_title,
    "keywords": sample_keywords
})
print(result_summary.content)

인공지능 기술은 머신러닝과 딥러닝의 발전을 통해 빠르게 진화하며 산업 전반에 혁신을 가져오고 있습니다. 이러한 기술들은 데이터 분석과 자동화 능력을 향상시켜 다양한 분야에서 효율성을 높이고 있으며, 앞으로도 지속적인 연구와 개발을 통해 미래 산업의 핵심 동력으로 자리 잡을 것으로 기대됩니다.


# ChatPromptTemplate를 이용해 역할 기반 프롬프트

In [12]:
from langchain.prompts import ChatPromptTemplate

In [17]:
# 시나리오 : 파이썬 프로그램 전문가 역할 부여, 사용자가 파이썬 관련 질문에 친절히, 간경한 답변 하도록
sys_message = (
            "당신은 파이썬 분야의 뛰어난 전문가이자 조언자입니다. "
            "사용자의 프로그리밍 질문에 재미있게 이해하기 쉽게 답변하십시오."
            )
user_message = "{question}" # 사용자 질문 -> 실행 시 입력값으로 채워진다.

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_message),
    ("user", user_message)
  ])

sample_question = "파이썬의 리스트 컴프리헨션(list comprehension)과 map 함수의 차이가 무엇이야?"

message = chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 파이썬 분야의 뛰어난 전문가이자 조언자입니다. 사용자의 프로그리밍 질문에 재미있게 이해하기 쉽게 답변하십시오.', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 리스트 컴프리헨션(list comprehension)과 map 함수의 차이가 무엇이야?', additional_kwargs={}, response_metadata={})]


In [18]:
result = (chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

아하! 리스트 컴프리헨션과 map 함수의 차이점을 궁금해하는군요! 둘 다 파이썬에서 반복 작업을 깔끔하게 처리하는 멋진 도구들이지만, 각각의 매력 포인트가 조금씩 달라요. 자, 재미있게 비교해볼까요?

1. **생김새와 문법의 차이!**
   - **리스트 컴프리헨션**은 마치 요리 레시피처럼 한 줄로 쓱! 만들어내는 느낌이에요.
     ```python
     squares = [x**2 for x in range(10)]
     ```
     이건 "0부터 9까지의 숫자를 제곱해서 리스트로 만들어라"라는 명령이죠.
   
   - **map 함수**는 좀 더 함수형 프로그래밍 스타일이에요. 함수를 인자로 넣어서 적용하는 방식이죠.
     ```python
     def square(x):
         return x**2
     squares = list(map(square, range(10)))
     ```
     또는 람다 함수와 함께 쓰면 더 간단하게!
     ```python
     squares = list(map(lambda x: x**2, range(10)))
     ```

2. **가독성과 직관성!**
   - 리스트 컴프리헨션은 한눈에 "이 리스트는 어떤 규칙으로 만들어졌는지" 알기 쉽고, 조건문도 넣기 좋아요.
     ```python
     even_squares = [x**2 for x in range(10) if x % 2 == 0]
     ```
   - map은 함수와 데이터를 따로 분리해서 생각하는 느낌이라, 복잡한 조건이 들어가면 조금 더 읽기 어려울 수 있어요.

3. **성능 차이?**
   - 둘 다 빠르지만, 내부 구현이 조금씩 달라서 미묘한 차이가 있을 수 있어요. 보통은 둘 다 충분히 빠르고, 가독성에 따라 선택하는 게 좋아요.

4. **언제 어떤 걸 쓸까?**
   - **리스트 컴프리헨션**은 간단한 변환이나 필터링에 딱! 좋아요.
   - **map**은 이미 만들어진 함수와

# PartialPromptTemplate

In [19]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [20]:
role_system_template = "당신은 {role} 분야의 전문 지식인입니다. 가능한 한 자세히 답변해 주세요."
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt = base_chat_prompt.partial(role="주식 투자")

In [21]:
sample_question = "현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은 어떻습니까?"
message = partial_chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인입니다. 가능한 한 자세히 답변해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은 어떻습니까?', additional_kwargs={}, response_metadata={})]


In [22]:
result = (partial_chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

2025년 5월 현재 시장 상황을 고려할 때, 삼성전자 주식의 전망은 여러 요인에 따라 달라질 수 있습니다. 아래에 주요 분석 포인트를 상세히 설명드리겠습니다.

1. 글로벌 경제 및 시장 환경
- 글로벌 경기 회복 여부: 2025년 현재 세계 경제는 일부 지역에서 회복세를 보이고 있으나, 미국, 중국, 유럽 등 주요 경제권의 성장률과 인플레이션, 금리 정책이 삼성전자 주가에 영향을 미칩니다.
- 금리 정책: 미국 연준과 한국은행의 금리 결정이 투자 심리와 자금 흐름에 영향을 미치며, 금리 인상은 기술주와 반도체 업종에 부정적일 수 있습니다.

2. 반도체 산업 동향
- 수요와 공급: 인공지능, 자율주행, 5G, IoT 등 첨단 기술 발전으로 반도체 수요는 지속적으로 증가하고 있으나, 공급 과잉이나 가격 하락 압력도 존재합니다.
- 기술 혁신: 삼성전자는 D램, 낸드플래시, 시스템 반도체 등 핵심 제품에서 경쟁력을 유지하고 있으며, 차세대 공정 기술 개발이 중요한 변수입니다.

3. 삼성전자의 내부 전략 및 재무 상태
- 신사업 및 투자: 인공지능, 6G, 차세대 디스플레이, 바이오 등 신사업에 대한 투자와 연구개발(R&D) 활동이 기업 성장에 기여할 수 있습니다.
- 재무 건전성: 안정적인 현금 흐름과 높은 수익성은 주가 안정성을 높이는 요인입니다.

4. 기술적 분석 및 시장 심리
- 주가 차트와 거래량: 최근의 주가 흐름, 지지선과 저항선, 이동평균선 등을 분석하면 단기적 또는 중장기적 방향성을 예측할 수 있습니다.
- 투자 심리: 글로벌 불확실성, 지정학적 리스크, 환율 변동 등도 시장 심리에 영향을 미칩니다.

5. 정책 및 규제 환경
- 정부 정책: 한국 정부의 반도체 산업 지원 정책, 글로벌 무역 정책, 규제 강화 여부도 중요한 변수입니다.

**전망 요약**  
2025년 5월 현재, 삼성전자는 글로벌 반도체 시장의 성장과 기술 혁신, 내부 전략의 성공 여부에 따라 긍정적 또는 부정적 영향을 받을 수 있습니다. 전반적으로, 반도체 산업의 장기 성